In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [3]:
df_fake['Label'] = 0
df_true['Label'] = 1

In [4]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [5]:
final_data = pd.concat([df_fake, df_true], axis=0)
final_data

,title,text,subject,date,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [6]:
final_data = final_data.sample(frac=1).reset_index(drop=True)

In [7]:
final_data.drop_duplicates(inplace=True)

In [8]:
final_data.duplicated().sum()

0

In [12]:
final_data=final_data[["title","Label"]]
final_data

,title,Label
0,HILLARY LIES AGAIN…She’s NOT The First Female ...,0
1,"Japan hails progress in TPP trade talks, but d...",1
2,SHOCKING VIDEO SHOWS HOW EASILY DRUG DEALERS S...,0
3,Greece passes sex change law opposed by Orthod...,1
4,DEMOCRAT MOCKS AMERICANS For Believing ‘Climat...,0
...,...,...
44893,USTR to seek 'appropriate mechanism' in NAFTA ...,1
44894,U.S.'s Mattis says eyeing provocative Iran act...,1
44895,Trump Might Have Just Put Himself In Prison W...,0
44896,GENERAL FLYNN: The Right Stuff…You’re In Good ...,0


In [13]:
import string

In [16]:
def wordpreprocessing(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

In [17]:
final_data['title']=final_data['title'].apply(wordpreprocessing)

In [18]:
final_data

,title,Label
0,hillary lies again she s not the first female ...,0
1,japan hails progress in tpp trade talks but d...,1
2,shocking video shows how easily drug dealers s...,0
3,greece passes sex change law opposed by orthod...,1
4,democrat mocks americans for believing climat...,0
...,...,...
44893,ustr to seek appropriate mechanism in nafta ...,1
44894,u s s mattis says eyeing provocative iran act...,1
44895,trump might have just put himself in prison w...,0
44896,general flynn the right stuff you re in good ...,0


In [20]:
import nltk

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize, sent_tokenize
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer 

In [35]:
X_train, X_test, y_train, y_test = train_test_split(final_data['title'], final_data['Label'], test_size=0.2, random_state=42)

In [36]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline

In [30]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")
df_manual=pd.DataFrame()
df_manual['title']=''
df_manual['text']=''
df_manual['subject']=''
df_manual['date']=''
df_manual['Label']=''


for i in range(516,526):
    df_manual.loc[len(df_manual)]= df_fake.loc[i]
    df_fake.drop(index=i, inplace=True, axis='index')
for i in range(487,498):
    df_manual.loc[len(df_manual)]= df_true.loc[i]
    df_true.drop(index=i, inplace=True, axis='index')
    
final_data = pd.concat([df_fake, df_true], axis=0)
final_data

final_data = final_data.sample(frac=1).reset_index(drop=True)
final_data.drop_duplicates(inplace=True)
final_data.duplicated().sum()

def reuters_removal(text):
    reuters_text=''
    reuters_text= text.split()
    reuters_text= [word for word in reuters_text if not word=="(Reuters)"]
    reuters_text= [word for word in reuters_text if not word=="Reuters"]
    reuters_text= " ".join(reuters_text)
    return reuters_text

final_data['text']=final_data['text'].apply(reuters_removal)
final_data.isnull().sum()

final_data=final_data[["text","Label"]]

port_stemmer= PorterStemmer()

import nltk
nltk.download('stopwords')

def stemmer(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ', content)
    stemmed_content= stemmed_content.lower()
    stemmed_content= stemmed_content.split()
    stemmed_content= [port_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content= ' '.join(stemmed_content)
    return stemmed_content

final_data['text']= final_data['text'].apply(stemmer)

max_len=100
data_text=final_data["text"]
data_label=final_data["Label"]

! pip install transformers

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.ops.numpy_ops import np_utils
from transformers import BertModel, TFBertModel 
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib import rcParams
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

X_train, X_test, Y_train, Y_test = train_test_split(data_text, data_label, stratify = data_label, test_size = 0.2, random_state =42)

def tokenize(X):
    
    X = bert_tokenizer(
        text = list(X),
        add_special_tokens = True,
        max_length = 100,
        truncation = True,
        padding = 'max_length',
        return_tensors = 'tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True
        )
    return X

X_train_token = tokenize(X_train)
X_test_token = tokenize(X_test)

maxlen=100


def create_model():
    dropout_rate=0.2
    input_ids=Input(shape=(maxlen,),dtype=tf.int32)
    input_mask=Input(shape=(maxlen,),dtype=tf.int32)
    bert_layer=bert_model([input_ids,input_mask])[1]
    x=Dropout(0.5)(bert_layer)
    x=Dense(64,activation="tanh")(x)
    x=Dropout(0.2)(x)
    x=Dense(1,activation="sigmoid")(x)
    model = Model(inputs=[input_ids, input_mask], outputs=x)
    return model
    
    
model=create_model()
model.summary()

optimizer = Adam(learning_rate=1e-05, epsilon=1e-08, decay=0.01,clipnorm=1.0)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = 'accuracy')

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max', verbose=1, patience=50,baseline=0.4,min_delta=0.0001,restore_best_weights=False)

history = model.fit(x = {'input_1':X_train_token['input_ids'],'input_2':X_train_token['attention_mask']}, y = Y_train, epochs=7, validation_split = 0.2, batch_size = 32, callbacks=[callback])